In [ ]:
'''
Data Collected from : https://en.tutiempo.net/climate/01-2011/ws-421810.html
'''

In [69]:
import os, time
import requests
from tqdm import tqdm
import pandas as pd
from datetime import datetime

In [70]:
def retrieve_html(verbose=False):
    for year in tqdm(range(2013, 2020)):
        for month in tqdm(range(1, 13), leave=False):
            if month < 10: month = '0'+str(month)
            global url
            url = f'https://en.tutiempo.net/climate/{month}-{year}/ws-421810.html'
#             print(url)
            url_text = requests.get(url)            
            text_utf = url_text.text.encode('utf-8')
            
            dir = f'database/html_data/{year}'
            fname = dir + f'/{month}.html'
            if not os.path.exists(dir):
                os.makedirs(dir)
            
            with open(fname, 'wb') as f:
                f.write(text_utf)

In [106]:
def url_to_df(url, year, month):

#     print(year, month)
    url = f'https://en.tutiempo.net/climate/{month}-{year}/ws-421810.html'
    url_text = requests.get(url)            
    text_utf = url_text.text.encode('utf-8')
    global df
    df = pd.read_html(text_utf)[2][:31] # this returns a list of all the tables in the page, but we only want a specific one with the index 2. Upto 31st row.

    
    for d in range(len(df.Day)):
        try:
            day = int(df.Day[d])
        except:
            break
#         timestamp = datetime(year, month, day).timestamp()
        timestamp = datetime(int(year), int(month), day)
        
        df.Day[d] = timestamp

    df.set_index('Day', inplace=True) 

    return df



def web_data_to_dataframe():
    master_df = pd.DataFrame()
    
    for year in tqdm(range(2013, 2020)):
        for month in tqdm(range(1, 13), leave=False):
            if month < 10: month = '0'+str(month) # converting the month format to sync with the url format
            url       = f'https://en.tutiempo.net/climate/{month}-{year}/ws-421810.html'
#             print(year, month)
            df        = url_to_df(url, year, month)
            master_df = pd.concat([master_df, df])
    
    return master_df

In [ ]:
if __name__ == '__main__':
    start = time.time()

    # Retrieving the Data
    df = web_data_to_dataframe()
    
    # Exporting the CSV
    df.to_csv(f'database/raw_master_data.csv')
    
    
    print('Time Taken :', time.time()-start, 'sec.')